# 数据转换
CSV 转为 Pickle 和 Feather 格式

In [1]:
import os
import pandas as pd

In [3]:
# 安全转换函数，避免数据错误导致读取失败
def safe_int(x):
    try:
        return int(x)
    except ValueError:
        return None

# 指定每列的数据类型，以优化读取速度
dtype = {
    'longitude': 'float32',  # 浮点型
    'latitude': 'float32',  # 浮点型
    'cog': 'float32',  # 浮点型
    'heading': 'float32',  # 浮点型
    'speed': 'float32',  # 浮点型
}

converters = {
    'mmsi': safe_int
}


In [4]:
# 指定主目录路径
directory_path = 'data/raw'  # 确保路径正确
output_path = 'data/'  # 合并后的文件路径

# 初始化一个空的DataFrame用于存放合并后的数据
df_combined = pd.DataFrame()

In [5]:
# 遍历所有文件夹和子文件夹
for root, dirs, files in os.walk(directory_path):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)

            # 先读取文件头以检查包含的列
            temp_df = pd.read_csv(file_path, nrows=0)
            if 'post_time' in temp_df.columns:
                parse_dates = ['post_time']

                # 读取CSV文件
                df = pd.read_csv(file_path, dtype=dtype, converters=converters, parse_dates=parse_dates,
                                 low_memory=False)
                # 根据具体情况选择必要的字段进行提取
                df = df[['mmsi', 'longitude', 'latitude', 'cog', 'speed', 'post_time']]
            elif 'ts' in temp_df.columns:
                parse_dates = ['ts']

                # 读取CSV文件
                df = pd.read_csv(file_path, dtype=dtype, converters=converters, parse_dates=parse_dates,
                                 low_memory=False)
                # 根据具体情况选择必要的字段进行提取和重命名
                df = df[['mmsi', 'longitude', 'latitude', 'heading', 'speed', 'ts']]
                df.rename(columns={'ts': 'post_time', 'heading': 'cog'}, inplace=True)

            else:
                print(f"Skipping file {file} as it does not contain 'post_time' or 'ts' columns.")
                continue

            # 合并数据到df_combined中
            df_combined = pd.concat([df_combined, df], ignore_index=True)

            print(f"Processed {file_path}")

# # 将post_time列转换为日期时间格式，并移除多余位数
df_combined['post_time'] = df_combined['post_time'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')

# 保存合并后的数据为Feather格式
df_combined.to_feather(output_path + 'combined_data.feather')
print(f"Combined data saved to {output_path + 'combined_data.feather'}")

# 保存合并后的数据为Pickel格式
df_combined.to_pickle(output_path + 'combined_data.pkl')
print(f"Combined data saved to {output_path + 'combined_data.pkl'}")

Processed data/raw\1. cq_tlx_device_1 export\1. cq_tlx_device_1 export\cq_tlx_device1_202405282229.csv
Processed data/raw\1. cq_tlx_device_1 export\1. cq_tlx_device_1 export\cq_tlx_device1_202405282229_2.csv
Processed data/raw\1. cq_tlx_device_1 export\1. cq_tlx_device_1 export\cq_tlx_device1_202405282229_3.csv
Processed data/raw\1. cq_tlx_device_1 export\1. cq_tlx_device_1 export\cq_tlx_device1_202405282229_4.csv
Processed data/raw\1. cq_tlx_device_1 export\1. cq_tlx_device_1 export\cq_tlx_device1_202405282229_5.csv
Processed data/raw\2. cq_tlx_dynamic_ais1 export\cq_tlx_dynamic_ais1_202405282248.csv
Processed data/raw\2. cq_tlx_dynamic_ais1 export\cq_tlx_dynamic_ais1_202405282248_2.csv
Processed data/raw\2. cq_tlx_dynamic_ais1 export\cq_tlx_dynamic_ais1_202405282248_3.csv
Processed data/raw\2. cq_tlx_dynamic_ais1 export\cq_tlx_dynamic_ais1_202405282248_4.csv
Processed data/raw\2024-05-28 保存所有重庆\ship_ais_all.csv
Combined data saved to data/combined_data.feather
Combined data saved to 

## 数据过滤

In [13]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
from shapely.strtree import STRtree
import numpy as np
from tqdm import tqdm

In [15]:
# 1. 读取整个PKL文件
def read_pkl_file(file_path):
    return pd.read_pickle(file_path)


# 2. 计算经纬度的四分位数范围，并进行初步过滤
def filter_by_quartiles(df):
    lon_quartiles = df['longitude'].quantile([0.25, 0.75])
    lat_quartiles = df['latitude'].quantile([0.25, 0.75])
    filtered = df[(df['longitude'] >= lon_quartiles[0.25]) & (df['longitude'] <= lon_quartiles[0.75]) &
                  (df['latitude'] >= lat_quartiles[0.25]) & (df['latitude'] <= lat_quartiles[0.75])]
    return filtered


# 3. 读取GeoJSON文件并建立空间索引
def read_geojson_file_with_spatial_index(file_path):
    geo_df = gpd.read_file(file_path)
    spatial_index = STRtree(geo_df.geometry)
    return geo_df, spatial_index


# 4. 计算点到最近几何对象的距离
def calculate_nearest_distance_with_index(df, geo_df, spatial_index):
    points = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
    points = points.set_crs(geo_df.crs, allow_override=True)  # Ensure the CRS matches

    def nearest_distance(point):
        nearest_geoms = spatial_index.query(point)
        if len(nearest_geoms) > 0:
            actual_geom = geo_df.geometry.iloc[nearest_geoms[0]]  # 根据索引取得几何对象
            return point.distance(actual_geom)  # 计算距离
        else:
            return np.nan  # 如果没有找到相应的几何对象，返回 NaN

    distances = np.array([nearest_distance(point) for point in points.geometry])
    df.loc[:, 'nearest_distance'] = distances  # 使用 .loc 显式指定修改操作
    return df


# 5. 根据指定距离范围进一步过滤数据
def filter_by_distance(df, max_distance):
    filtered_df = df[df['nearest_distance'] <= max_distance].dropna(subset=['nearest_distance'])
    return filtered_df

def process_data(pkl_file_path, geojson_file_path, max_distance, chunksize=100000):
    geo_df, spatial_index = read_geojson_file_with_spatial_index(geojson_file_path)

    # 读取整个PKL文件
    df = read_pkl_file(pkl_file_path)

    # 分块处理数据
    num_chunks = len(df) // chunksize + 1
    filtered_chunks = []

    # 使用 tqdm 包装循环以显示进度条
    for i in tqdm(range(num_chunks), desc="Processing chunks"):
        chunk = df[i * chunksize:(i + 1) * chunksize]
        filtered_chunk = filter_by_quartiles(chunk)
        # filtered_chunk = calculate_nearest_distance_with_index(filtered_chunk, geo_df, spatial_index)
        # filtered_chunk = filter_by_distance(filtered_chunk, max_distance)
        filtered_chunks.append(filtered_chunk)

    # 合并所有处理过的分块
    final_df = pd.concat(filtered_chunks, ignore_index=True)
    return final_df

In [16]:
# 调用函数
pkl_file_path = 'data/combined_data.pkl'
geojson_file_path = 'data/geojson/500108.geojson'
max_distance = 100  # 例如 100 米以内
output_path = 'data/filtered_data.pkl'

filtered_data = process_data(pkl_file_path, geojson_file_path, max_distance)
print(filtered_data.head())

filtered_data.to_pickle(output_path)
print(f"Filtered data saved to {output_path}")

Processing chunks: 100%|██████████| 178/178 [00:00<00:00, 189.55it/s]


          mmsi   longitude   latitude     cog  speed  \
0  413870065.0  106.631256  29.595894     0.0    0.0   
1  413941058.0  106.633766  29.589529  3600.0    0.0   
2  413826177.0  106.650078  29.596764  3472.0    0.0   
3  413825495.0  106.626350  29.599640     0.0    0.0   
4  413762537.0  106.629311  29.592377  1849.0    0.0   

                    post_time  
0  2023-05-21 15:53:13.000000  
1  2023-05-21 15:53:16.000000  
2  2023-05-21 15:53:17.000000  
3  2023-05-21 15:53:20.000000  
4  2023-05-21 15:53:21.000000  
Filtered data saved to data/filtered_data.pkl


In [22]:
filtered_data.head(100)

,mmsi,longitude,latitude,cog,speed,post_time,time_diff,new_segment,segment_id
44981,0.0,106.651237,29.594469,911.0,72.0,2023-05-23 15:11:57,NaT,True,1
929698,0.0,106.627975,29.594999,0.0,0.0,2023-08-28 15:45:45,97 days 00:33:48,True,2
1120640,0.0,106.647430,29.591801,701.0,254.0,2023-09-06 10:18:17,8 days 18:32:32,True,3
1122080,0.0,106.649590,29.592422,2475.0,147.0,2023-09-06 11:48:05,0 days 01:29:48,True,4
1122213,0.0,106.627922,29.594973,0.0,0.0,2023-09-06 12:00:05,0 days 00:12:00,False,4
...,...,...,...,...,...,...,...,...,...
3703502,18907.0,106.644302,29.591883,2826.0,79.0,2024-05-28 20:00:57,0 days 00:00:31,False,8
3703515,18907.0,106.643066,29.591990,2624.0,80.0,2024-05-28 20:01:26,0 days 00:00:29,False,8
3703526,18907.0,106.641670,29.591640,2553.0,89.0,2024-05-28 20:01:59,0 days 00:00:33,False,8
3703552,18907.0,106.637306,29.590767,2825.0,98.0,2024-05-28 20:03:27,0 days 00:01:28,False,8


In [3]:
import pandas as pd

# 加载数据
filtered_data = pd.read_pickle('data/filtered_data.pkl')
filtered_data['post_time'] = pd.to_datetime(filtered_data['post_time'])
filtered_data.sort_values(['mmsi', 'post_time'], inplace=True)

# 计算每个数据点与前一个数据点的时间差
filtered_data['time_diff'] = filtered_data.groupby('mmsi')['post_time'].diff()

# 标记超过30分钟的时间差为新轨迹段的开始
filtered_data['new_segment'] = filtered_data['time_diff'] > pd.Timedelta(minutes=30)

# 确保第一个数据点每个mmsi也被标记为新轨迹的开始
filtered_data.loc[0, 'new_segment'] = True
filtered_data['new_segment'] = filtered_data['new_segment'].fillna(True)

# 生成轨迹段ID
filtered_data['segment_id'] = filtered_data['new_segment'].cumsum()


In [4]:
filtered_data.head(10)


,mmsi,longitude,latitude,cog,speed,post_time,time_diff,new_segment,segment_id
44981,0.0,106.651237,29.594469,911.0,72.0,2023-05-23 15:11:57.000,NaT,False,0
929698,0.0,106.627975,29.594999,0.0,0.0,2023-08-28 15:45:45.000,97 days 00:33:48,True,1
1120640,0.0,106.647430,29.591801,701.0,254.0,2023-09-06 10:18:17.000,8 days 18:32:32,True,2
1122080,0.0,106.649590,29.592422,2475.0,147.0,2023-09-06 11:48:05.000,0 days 01:29:48,True,3
1122213,0.0,106.627922,29.594973,0.0,0.0,2023-09-06 12:00:05.000,0 days 00:12:00,False,3
1122310,0.0,106.627953,29.594984,0.0,0.0,2023-09-06 12:06:04.000,0 days 00:05:59,False,3
1122424,0.0,106.627785,29.596140,3500.0,76.0,2023-09-06 12:12:03.000,0 days 00:05:59,False,3
1168988,0.0,106.627274,29.598040,3471.0,162.0,2023-09-08 09:13:12.000,1 days 21:01:09,True,4
844500,18907.0,106.628098,29.597054,1680.0,84.0,2023-08-23 16:23:22.000,NaT,False,4
4638141,18907.0,106.628716,29.595263,1623.0,53.0,2023-08-23 16:24:21.732,0 days 00:00:59.732000,False,4


In [6]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6954479 entries, 44981 to 1434036
Data columns (total 9 columns):
 #   Column       Dtype          
---  ------       -----          
 0   mmsi         float64        
 1   longitude    float32        
 2   latitude     float32        
 3   cog          float32        
 4   speed        float32        
 5   post_time    datetime64[ns] 
 6   time_diff    timedelta64[ns]
 7   new_segment  bool           
 8   segment_id   int64          
dtypes: bool(1), datetime64[ns](1), float32(4), float64(1), int64(1), timedelta64[ns](1)
memory usage: 636.0 MB


In [5]:
filtered_data.to_pickle('data/segmented_data.pkl')